# Preprocessing

In [15]:
%matplotlib inline
import pandas
import matplotlib.pyplot as plt
from pandas import plotting
import numpy as np

pandas.set_option('display.max_columns', None)  
pandas.set_option('display.expand_frame_repr', False)
pandas.set_option('max_colwidth', -1)
pandas.set_option('display.precision', 3)

data = pandas.read_csv('../datasets/SpotifyAudioFeatures-full-29Nov.csv', index_col=0)
data['duration'] = data['duration_ms']*(1/1000)#*(1/60)
#data = data.drop(labels='Unnamed: 0',axis=1)
data = data.drop(labels='duration_ms',axis=1)
data = data.drop(labels='artist_name',axis=1)
data = data.drop(labels='track_name',axis=1)
data = data.drop(labels='track_id',axis=1)
data.head()

,popularity,num_artists,release_month,release_weekday,artist_followers,artist_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_len,artist_len,duration
0,99,1,9,5,19671232,100,0.695,0.762,0,-3.497,1,0.040,0.192,2.440e-03,0.086,0.553,120.042,4,7,11,215.280
1,98,1,10,5,7153061,97,0.598,0.427,7,-8.764,0,0.032,0.055,5.830e-06,0.210,0.060,76.469,4,19,12,175.721
2,94,2,10,5,6183091,94,0.474,0.631,5,-5.884,0,0.343,0.069,0.000e+00,0.132,0.425,180.051,4,40,10,189.323
3,99,1,10,3,17952498,93,0.505,0.340,4,-9.005,1,0.044,0.576,0.000e+00,0.210,0.092,101.993,4,19,12,206.459
4,97,1,8,5,2549258,89,0.778,0.559,6,-7.109,0,0.078,0.128,0.000e+00,0.399,0.678,129.989,1,15,9,185.093


In [7]:
import sklearn
from sklearn import tree
from sklearn.model_selection import GridSearchCV
import sklearn.model_selection as cv

X = data.drop(labels='popularity', axis=1)

for i in [2,3,4]:
    y = pandas.qcut(data['popularity'],i,labels=[x for x in range(i)])
    (X_train, X_test, y_train, y_test) = cv.train_test_split(X, y, test_size=.3, random_state=1)
    
    params = {'min_impurity_decrease': list(np.linspace(0,0.5,21)),'min_samples_split':list(range(2,20,11))}
    clf = GridSearchCV(tree.DecisionTreeClassifier(criterion='entropy'), param_grid=params,cv=10,n_jobs=-1)  # If cv is integer, by default is Stratifyed 
    clf.fit(X_train, y_train)
    #print("Best Params=",clf.best_params_, "Accuracy=", clf.best_score_)
    clf=tree.DecisionTreeClassifier(criterion='entropy', min_samples_split=clf.best_params_['min_samples_split'],min_impurity_decrease=clf.best_params_['min_impurity_decrease'])
    clf = clf.fit(X_train, y_train)
    pred = clf.predict(X_test)

    # Obtain accuracy score of learned classifier on test data
    print("Accuracy of",i, "classes" ":", sklearn.metrics.accuracy_score(y_test, pred))
    epsilon = sklearn.metrics.accuracy_score(y_test, pred)


Accuracy of 2 classes: 0.7384969325153374
Accuracy of 3 classes: 0.6687116564417178
Accuracy of 4 classes: 0.5851226993865031


In [16]:
print(pandas.qcut(data['popularity'], 2))
print(pandas.qcut(data['popularity'], 3))
print(pandas.qcut(data['popularity'], 4))


0       (61.0, 100.0] 
1       (61.0, 100.0] 
2       (61.0, 100.0] 
3       (61.0, 100.0] 
4       (61.0, 100.0] 
            ...       
8687    (-0.001, 61.0]
8688    (-0.001, 61.0]
8689    (61.0, 100.0] 
8690    (61.0, 100.0] 
8691    (-0.001, 61.0]
Name: popularity, Length: 8692, dtype: category
Categories (2, interval[float64]): [(-0.001, 61.0] < (61.0, 100.0]]
0       (65.0, 100.0] 
1       (65.0, 100.0] 
2       (65.0, 100.0] 
3       (65.0, 100.0] 
4       (65.0, 100.0] 
            ...       
8687    (-0.001, 57.0]
8688    (-0.001, 57.0]
8689    (65.0, 100.0] 
8690    (65.0, 100.0] 
8691    (57.0, 65.0]  
Name: popularity, Length: 8692, dtype: category
Categories (3, interval[float64]): [(-0.001, 57.0] < (57.0, 65.0] < (65.0, 100.0]]
0       (67.0, 100.0]
1       (67.0, 100.0]
2       (67.0, 100.0]
3       (67.0, 100.0]
4       (67.0, 100.0]
            ...      
8687    (50.0, 61.0] 
8688    (50.0, 61.0] 
8689    (67.0, 100.0]
8690    (67.0, 100.0]
8691    (50.0, 61.0] 
Name:

In [13]:
# We want to transform popularity to 2 classes
print(pandas.qcut(data['popularity'], 2))
data['popularity'] = pandas.qcut(data['popularity'], 2,labels=[0,1])

print("Percentage of examples of the smaller class (class 1): {0:.2f}%".format(100*data[data.popularity == 1].count()[0]/(data[data.popularity == 0].count()[0]+data[data.popularity == 1].count()[0])))

0       (61.0, 100.0] 
1       (61.0, 100.0] 
2       (61.0, 100.0] 
3       (61.0, 100.0] 
4       (61.0, 100.0] 
            ...       
8687    (-0.001, 61.0]
8688    (-0.001, 61.0]
8689    (61.0, 100.0] 
8690    (61.0, 100.0] 
8691    (-0.001, 61.0]
Name: popularity, Length: 8692, dtype: category
Categories (2, interval[float64]): [(-0.001, 61.0] < (61.0, 100.0]]
Percentage of examples of the smaller class (class 1): 47.24%


## Description of numeric values

In [4]:
data.describe()

,num_artists,release_month,release_weekday,artist_followers,artist_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_len,artist_len,duration
count,8692.000,8692.000,8692.000,8.692e+03,8692.000,8692.000,8.692e+03,8692.000,8692.000,8692.000,8692.000,8692.000,8.692e+03,8692.000,8692.000,8692.000,8692.000,8692.000,8692.000,8692.000
mean,1.788,7.976,4.605,2.496e+06,74.684,0.576,5.030e-01,5.083,-10.596,0.643,0.112,0.418,1.815e-01,0.177,0.440,117.307,3.861,26.324,12.088,211.149
std,1.202,3.163,1.068,5.313e+06,10.876,0.206,2.735e-01,3.569,7.578,0.479,0.121,0.378,3.428e-01,0.144,0.240,32.037,0.539,25.483,5.474,107.919
min,1.000,1.000,1.000,0.000e+00,0.000,0.000,2.020e-05,0.000,-43.710,0.000,0.000,0.000,0.000e+00,0.000,0.000,0.000,0.000,1.000,2.000,6.062
25%,1.000,5.000,5.000,1.733e+05,69.000,0.430,2.660e-01,2.000,-14.428,0.000,0.040,0.065,0.000e+00,0.098,0.245,91.987,4.000,9.000,8.000,161.584
50%,1.000,9.000,5.000,1.032e+06,77.000,0.603,5.575e-01,5.000,-7.331,1.000,0.055,0.281,7.145e-06,0.119,0.427,116.303,4.000,16.000,11.000,195.007
75%,2.000,11.000,5.000,2.943e+06,81.000,0.738,7.190e-01,8.000,-5.388,1.000,0.129,0.855,5.047e-02,0.197,0.616,139.983,4.000,34.000,15.000,229.188
max,16.000,11.000,7.000,5.466e+07,100.000,0.981,1.000e+00,11.000,0.634,1.000,0.951,0.996,9.980e-01,0.986,0.980,241.005,5.000,233.000,49.000,1547.104


## Identify errors or ouliers - univariate analysis

In [7]:
data[data.duration>data.duration.mean()+5*data.duration.std()]

,popularity,num_artists,release_month,release_weekday,artist_followers,artist_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_len,artist_len,duration
1180,1,1,8,5,1032672,79,0.461,0.566,9,-9.133,0,0.057,9.890e-03,0.786,0.109,0.387,88.067,1,10,4,764.457
1568,1,1,8,5,1032672,79,0.323,0.803,7,-7.397,1,0.037,7.350e-05,0.834,0.096,0.386,144.755,5,7,4,943.529
1684,1,1,8,5,1032672,79,0.330,0.610,10,-9.571,1,0.049,2.190e-03,0.769,0.107,0.240,172.677,1,10,4,817.832
1833,0,1,11,5,3887175,83,0.326,0.898,2,-5.658,1,0.075,7.170e-01,0.877,0.227,0.356,109.044,4,37,13,785.320
4667,0,2,11,5,2350327,80,0.310,0.113,2,-15.995,0,0.043,9.950e-01,0.536,0.107,0.453,87.473,3,63,21,920.680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7660,0,4,11,5,2942677,77,0.259,0.111,0,-22.765,1,0.037,9.850e-01,0.877,0.069,0.165,69.406,4,71,20,1055.000
7689,0,9,11,5,2942677,77,0.289,0.222,0,-16.821,1,0.070,9.440e-01,0.001,0.096,0.156,82.209,3,233,20,913.267
7706,0,4,11,5,2942677,77,0.358,0.139,3,-18.998,1,0.036,9.700e-01,0.869,0.077,0.115,131.154,4,77,20,1253.333
7726,0,4,11,5,2942677,77,0.280,0.083,2,-21.032,1,0.045,9.430e-01,0.575,0.068,0.103,111.448,4,72,20,1463.293


We can see above some very long songs (more than 10 minutes). Since there are a lot of them we can assure they aren't outliers or errors.

### Common error checkups

In [8]:
data[data.release_month > 12]
data[data.release_weekday > 7]
data[data.num_artists < 1]
data.isnull().any()

popularity           False
num_artists          False
release_month        False
release_weekday      False
artist_followers     False
artist_popularity    False
danceability         False
energy               False
key                  False
loudness             False
mode                 False
speechiness          False
acousticness         False
instrumentalness     False
liveness             False
valence              False
tempo                False
time_signature       False
track_len            False
artist_len           False
duration             False
dtype: bool

## Normalization/Standarization

We first create our labels by dividing the 'popularity' feature into three categories. With that we try a simple knn classifier and see its score. We then check the range of each numerical feature to see if a low score can be attributed to the data not being normalized.

### Obtaining labels

In [10]:
dat = data.apply(pandas.to_numeric)
#dat['popularity'] = pandas.qcut(dat['popularity'], 3, labels=[1, 2, 3]) # 1=not popular, 2=popular, 3=very popular

X_data = dat.drop(labels='popularity',axis=1).values #to_numpy()
Y_data = dat['popularity'].values #to_numpy()

In [11]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import sklearn.neighbors as nb
import numpy as np

cv = StratifiedKFold(n_splits=10, random_state=1) 

cv_scores = cross_val_score(nb.KNeighborsClassifier(), X=X_data,y=Y_data, cv=cv, scoring='accuracy')  
np.mean(cv_scores)

0.6861947137030564

In [12]:
from sklearn.model_selection import GridSearchCV
params = {'n_neighbors':list(range(1,20,2)), 'weights':('distance','uniform')}
knc = nb.KNeighborsClassifier()
cv = StratifiedKFold(n_splits=10, random_state=1) 
clf = GridSearchCV(knc, param_grid=params,cv=cv,n_jobs=-1,iid='False')  # If cv is integer, by default is Stratifyed 
clf.fit(X_data, Y_data)
print("Best Params=",clf.best_params_, "Accuracy=", clf.best_score_)

Best Params= {'n_neighbors': 7, 'weights': 'distance'} Accuracy= 0.7115738610216291


In [13]:
for i in range(len(X_data[1,:])):
    print('std:',X_data[:,i].std(),'min',X_data[:,i].min(),'max',X_data[:,i].max())

std: 1.20205437745497 min 1.0 max 16.0
std: 3.1626219260245687 min 1.0 max 11.0
std: 1.0677083151229125 min 1.0 max 7.0
std: 5312538.312650487 min 0.0 max 54656985.0
std: 10.875554160301865 min 0.0 max 100.0
std: 0.20610144805174987 min 0.0 max 0.981
std: 0.27344366410409376 min 2.02e-05 max 1.0
std: 3.568549823179129 min 0.0 max 11.0
std: 7.5775726818380305 min -43.71 max 0.634
std: 0.479181916635648 min 0.0 max 1.0
std: 0.12116329851382378 min 0.0 max 0.951
std: 0.37815291397229045 min 0.0 max 0.996
std: 0.3427808101655043 min 0.0 max 0.998
std: 0.14438707068518852 min 0.0 max 0.986
std: 0.23981047734195707 min 0.0 max 0.98
std: 32.03551877978578 min 0.0 max 241.005
std: 0.5394045374014891 min 0.0 max 5.0
std: 25.481160619887188 min 1.0 max 233.0
std: 5.473355858118832 min 2.0 max 49.0
std: 107.91271756288783 min 6.062 max 1547.104


In [14]:
from sklearn import preprocessing

# One way is to standarize all data mean 0, std 1
scaler = preprocessing.StandardScaler().fit(X_data)
# scaler = preprocessing.MinMaxScaler().fit(X_data)
X2=scaler.transform(X_data)
#X2 = preprocessing.scale(X_data)

for i in range(13):
    print('std:',X2[:,i].std(),'min',X2[:,i].min(),'max',X2[:,i].max())

std: 1.0 min -0.6553246374131798 max 11.823312170813837
std: 1.0 min -2.2057141245165575 max 0.956219307270545
std: 1.0 min -3.3765346899393363 max 2.2429776010268454
std: 1.0 min -0.46991539102468605 max 9.818384058947846
std: 1.0 min -6.867119694932162 max 2.3278140552046094
std: 1.0000000000000002 min -2.795316878487137 max 1.964475006884194
std: 1.0 min -1.8395771535074206 max 1.8174093898682213
std: 1.0 min -1.4242773523737737 max 1.6582072828555543
std: 0.9999999999999999 min -4.37002997300971 max 1.4819759214746746
std: 1.0 min -1.3414007147816598 max 0.7454893895466356
std: 1.0 min -0.9236985381274513 max 6.925212903413556
std: 1.0 min -1.1051018692337735 max 1.5287532808893933
std: 0.9999999999999999 min -0.5296325624214641 max 2.381848974477107


In [15]:
# Let's repeat the 10-fold Cross-validation with new data X2
knc = nb.KNeighborsClassifier()
cv = StratifiedKFold(n_splits=10, random_state=1) 
clf = GridSearchCV(knc, param_grid=params,cv=cv,n_jobs=-1,iid='False')  # If cv is integer, by default is Stratifyed 
clf.fit(X2, Y_data)
print("Best Params=",clf.best_params_, "Accuracy=", clf.best_score_)

Best Params= {'n_neighbors': 17, 'weights': 'distance'} Accuracy= 0.7344684767602393


In [16]:
#norm = pandas.DataFrame(Y_data, X2)
norm_df = dat.drop(labels='popularity',axis=1)
norm = scaler.fit_transform(norm_df)
norm_df = pandas.DataFrame(norm, index=norm_df.index, columns=norm_df.columns)
norm_df['popularity'] = dat['popularity'].values
norm_df.to_csv('../datasets/NormalizedSpotifyDataset.csv')
data.to_csv('../datasets/SpotifyDataset.csv')

In [17]:
df = pandas.read_csv('../datasets/NormalizedSpotifyDataset.csv', index_col=0)
df.head()

,num_artists,release_month,release_weekday,artist_followers,artist_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_len,artist_len,duration,popularity
0,-0.655,0.324,0.370,3.233,2.328,0.577,0.947,-1.424,0.937,0.745,-0.598,-0.597,-0.523,-0.627,0.473,0.085,0.259,-0.758,-0.199,0.038,1
1,-0.655,0.640,0.370,0.877,2.052,0.106,-0.278,0.537,0.242,-1.341,-0.662,-0.961,-0.530,0.230,-1.581,-1.275,0.259,-0.287,-0.016,-0.328,1
2,0.177,0.640,0.370,0.694,1.776,-0.495,0.468,-0.023,0.622,-1.341,1.907,-0.923,-0.530,-0.310,-0.061,1.959,0.259,0.537,-0.381,-0.202,1
3,-0.655,0.640,-1.503,2.909,1.684,-0.345,-0.596,-0.303,0.210,0.745,-0.562,0.418,-0.530,0.230,-1.451,-0.478,0.259,-0.287,-0.016,-0.043,1
4,-0.655,0.008,0.370,0.010,1.316,0.980,0.205,0.257,0.460,-1.341,-0.283,-0.767,-0.530,1.539,0.994,0.396,-5.303,-0.444,-0.564,-0.241,1
